In [1]:
import torch
from torch import nn

import torch.utils.data
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda")

batch_size = 64

kwargs = {'num_workers': 4, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/tmp/data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [3]:
from torch import nn, optim
from torch.nn import functional as F

log_interval = 1000
batch_size   = 100

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, 100):
    train(epoch)
    #test(epoch)
    #with torch.no_grad():
    #    sample = torch.randn(64, 20).to(device)
    #    sample = model.decode(sample).cpu()
    #    save_image(sample.view(64, 1, 28, 28),
    #               'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 547.179443
====> Epoch: 1 Average loss: 146.3342
Train Epoch: 2 [0/60000 (0%)]	Loss: 115.782394
====> Epoch: 2 Average loss: 115.3000
Train Epoch: 3 [0/60000 (0%)]	Loss: 118.031769
====> Epoch: 3 Average loss: 111.0128
Train Epoch: 4 [0/60000 (0%)]	Loss: 113.830551
====> Epoch: 4 Average loss: 109.0829
Train Epoch: 5 [0/60000 (0%)]	Loss: 106.127304
====> Epoch: 5 Average loss: 107.9462
Train Epoch: 6 [0/60000 (0%)]	Loss: 106.170776
====> Epoch: 6 Average loss: 107.1602
Train Epoch: 7 [0/60000 (0%)]	Loss: 109.772820
====> Epoch: 7 Average loss: 106.5870
Train Epoch: 8 [0/60000 (0%)]	Loss: 101.208199
====> Epoch: 8 Average loss: 106.1003
Train Epoch: 9 [0/60000 (0%)]	Loss: 110.027084
====> Epoch: 9 Average loss: 105.7619
Train Epoch: 10 [0/60000 (0%)]	Loss: 104.114006
====> Epoch: 10 Average loss: 105.3907
Train Epoch: 11 [0/60000 (0%)]	Loss: 108.506729
====> Epoch: 11 Average loss: 105.1232
Train Epoch: 12 [0/60000 (0%)]	Loss: 101.528526
====> Epoch: 